In [ ]:
from ultralytics import YOLO
from matplotlib import pyplot as plt
from PIL import Image

In [ ]:
#Instance
model =  YOLO("yolo11n-seg.yaml")
model = YOLO("yolo11n-seg.pt")
model =  YOLO("yolo11n-seg.yaml").load("yolo11n-seg.pt")

In [ ]:
# define number of classes based on YAML
import yaml
with open("yolo_dataset/data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

In [ ]:
#Define a project --> Destination directory for all results
project = "yolo_dataset/results"
#Define subdirectory for this specific training
name = "5_epochs-" #if the training again runs again, it creates a directory: 5_epochs-2

In [ ]:
%load_ext tensorboard
%tensorboard --logdir "yolo_dataset/results/5_epochs-" --port 6006


In [ ]:
# Train the model
results = model.train(data='yolo_dataset/data.yaml',
                      project=project,
                      name=name,
                      epochs=5,
                      patience=0, 
                      batch=4,
                      imgsz=512
                     )

In [ ]:
### Prediction (Gradio)
import gradio as gr
from ultralytics import YOLO

model = YOLO("yolo_dataset/results/5_epochs-/weights/last.pt")

def predict_image(img, conf_threshold, iou_threshold):
    results = model.predict(
        source=img,
        conf=conf_threshold,
        iou=iou_threshold,
        show_labels=True,
        show_conf=True,
    )
    return results[0].plot() if results else None

iface = gr.Interface(
    fn=predict_image,
    inputs=[
        gr.Image(type="pil", label="Upload Image"),
        gr.Slider(minimum=0, maximum=1, value=0.25, label="Confidence threshold"),
        gr.Slider(minimum=0, maximum=1, value=0.45, label="IoU threshold"),
    ],
    outputs=gr.Image(type="pil", label="Result"),
    title="Gradio YOLO11 Nuclei Segmentation",
    description="Upload images of H&E nuclei for YOLO11 detection.",
)
iface.launch(share=True)